<div style="background-color: #ADD8E6; border: 1px solid gray; padding: 3px">
    <h3>Release Planning Sample Workflow</h3>
    The following is an overview of the workflow:
    <ul>
    <li><b>Release Planning</b>: Prepares backlog of issues.</li>
    <li><b>Sprint Planning Cycle</b>: Scopes out sprint (includes Go/No-Go Decision for readiness. If readiness passes, signals kickoff of      next backlog issue, or ENDS if there are no more issues.) Also finetunes sprint tasks by preparing technical documents and plans which will be used as "goldens" for the sprint.</li>
    <li><b>Sprint Implementation Cycle</b>: Iterative implementation of the code,
    documents and/or tests that were indicated/"specified" by the specs in
    the planning phase.</li>
    <li><b>Sprint Evaluation Cycle</b>: Evaluates the code, documents, tests etc.
    created during the implementation cycle.
    </li>
    <li><b>Release Engineering</b>: Prepares the artifacts for readiness review.</li>
    <li><b>Release Delivery</b>: Delivers release.
    </li>
    </ul>
</div>

## Release Initiation / Kickoff

In [1]:
##############################################################################
# Imports
##############################################################################
import importlib
from crewai import Agent, Task, Crew, Process, LLM
from crewai.project import CrewBase, agent, crew, task
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List, Optional
from pydantic import Field, BaseModel
from crewai_tools import SerperDevTool
from crewai.tools import tool
from crewai.flow.flow import Flow, listen, start, router
from crewai.flow.persistence import persist
import os
from urllib.parse import urlparse
from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))
from tools import github_util
import logging
import traceback
import shutil
import time
from examples.crew import ReleaseCycle, SprintCycle
logging.basicConfig(level=logging.INFO)
import asyncio
import nest_asyncio
nest_asyncio.apply()

/opt/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tolaawofolu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tolaawofolu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/tolaawofolu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tolaawofolu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/tolaawofolu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading pa

In [2]:
##########################################################
# Instance Variables
##########################################################
_GITHUB_PROJECT = os.getenv("GITHUB_PROJECT")

_ORIGINAL_SPEC = os.getenv("ORIGINAL_SPEC")

_FEATURE_BRANCH = "template"

_BASE_BRANCH = "refactored"

_LOCAL_PATH = "tmp"


In [3]:
@persist()
class ReleaseInitiationFlow(Flow):
    """Flow for ReleaseInitiation."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        self.state['retries'] = 0
        
        self.max_depth = 10
    
    @start()
    def setup_environment(self):

        logging.info(f"✅ Starting flow {self.state['id']}...cloning codebase and creating new feature branch...")
        
        github_util.clone_repo(branch=_BASE_BRANCH, local_path=_LOCAL_PATH)
        
        github_util.create_feature_branch(_FEATURE_BRANCH, local_path=_LOCAL_PATH)
        
    
    @router(setup_environment)
    def start_release_planning(self):

        logging.info(f"Starting flow {self.state['id']}...checking if release has any issues...")

        has_no_issues = github_util.is_project_empty(_GITHUB_PROJECT)

        if has_no_issues:

            logging.info(f"Board has no issues. Creating user stories...")

            additional_context = self.state['additional_context']

            if additional_context:

                additional_context = f"For additional context, read from this single file:\n{additional_context}"
                

            output = ReleaseCycle(_FEATURE_BRANCH, "REFERENCE").crew().kickoff(inputs={"input": self.state['input'],

                                                            "additional_context": additional_context,

                                                            "output_base_path": f"{_LOCAL_PATH}"})

            if not output.pydantic:
                
                logging.error("Could not convert user stories into issues.")

                return "end"

            logging.info("Converting user stories into issues...")

            for story in output.pydantic.stories:
                
                github_util.create_issue(title=story.title, 
                             
                             body=f"{story.body}\n\nAcceptance Criteria:\n{story.acceptance_criteria}", 
                             
                             project=_GITHUB_PROJECT)
        
        return "check_should_sprint_continue"
        
    @router("check_should_sprint_continue")
    def should_sprint_continue(self):
        """
        Determines if the sprint should continue and processes accordingly.

        NOTE: This is NOT threadsafe; should be OK if issues are processed one at a time.
        """

        logging.info(f"Checking if sprint can proceed...")

        is_blocked = github_util.is_sprint_blocked(_GITHUB_PROJECT)
        
        is_in_progress = github_util.is_sprint_in_progress(_GITHUB_PROJECT)

        logging.info(f"is_blocked={is_blocked}, is_in_progress={is_in_progress}")

        if is_blocked:

            logging.info("################################"
            "❌ Board is BLOCKED! exiting..."
            "################################")

            return "end"

        elif is_in_progress:

            self.state['retries'] += 1

            if self.state['retries'] >= self.max_depth:

                logging.info("❌ Retries exhausted; exiting.")

                return "end"

            else:

                logging.info(f"Existing issue is being processed...")

                self.state["sprint_update_delay"] = '60'

                return "wait_for_sprint_update"
                             
        else:

            self.state['retries'] = 0

            logging.info("No existing issue is being processed. Processing the next issue in the backlog...")

            data = github_util.get_top_issue_in_status(status_name="Backlog", project=_GITHUB_PROJECT)
    
            next_issue = github_util.get_issue_by_number(data.get("issue_number"), project = _GITHUB_PROJECT)

            if next_issue:

                logging.info(f"Moving next issue from <Backlog> to <In progress>...")

                github_util.move_top_issue_to_status(old_status="Backlog", new_status="In progress", project = _GITHUB_PROJECT)

                logging.info(f"Kicking off issue #{str(next_issue["number"])}...")

                output = SprintCycle(_FEATURE_BRANCH, "REFERENCE", category="doc").crew().kickoff(
                    inputs={"input": next_issue["body"],
                            "output_base_path": f"{_LOCAL_PATH}"})
                
            logging.info("✅ Moving to next issue if it exists...")

            self.state["sprint_update_delay"] = '5'
            
            return "wait_for_sprint_update"

    
    @router("wait_for_sprint_update")
    def deliver_sprint_update(self):

        logging.info("Received 'wait_for_sprint_update'...")  

        delay = self.state["sprint_update_delay"] or '5'

        time.sleep(int(delay))

        logging.info("Returning 'check_should_sprint_continue'...")  

        return "check_should_sprint_continue"

                
    @listen("end")
    def end(self):

        logging.info("################################"
        "✅ THE END! Workflow complete."
        "################################")

### Execute Release Initiation Flow
Execute the flow!

In [4]:
##############################################################################
# Execute the Flow
##############################################################################

flow = ReleaseInitiationFlow()

flow.plot("ReleaseInitiationFlowPlot")

result = flow.kickoff(inputs={"input": f"./{_LOCAL_PATH}/{_ORIGINAL_SPEC}",
                              "additional_context": f"{_LOCAL_PATH}/README.md"})

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: ReleaseInitiationFlow                                                                                    │
│  ID: 7c42fa2d-193c-4517-bb3c-79cc3d810fbc                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 Flow started with ID: 7c42fa2d-193c-4517-bb3c-79cc3d810fbc

INFO:crewai.flow.flow:Flow started with ID: 7c42fa2d-193c-4517-bb3c-79cc3d810fbc


/opt/miniconda3/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

INFO:root:✅ Starting flow 7c42fa2d-193c-4517-bb3c-79cc3d810fbc...cloning codebase and creating new feature branch...
INFO:root:Cloning repo to tmp...
INFO:root:Repo tmp already exists. Removing...
INFO:root:Repo name: agapebondservant/industrial-data-project


...

INFO:root:Checking if sprint can proceed...
INFO:root:Repo name: agapebondservant/industrial-data-project
INFO:root:Repo name: agapebondservant/industrial-data-project
INFO:root:Project 'Code Understanding' has 0 active issue(s).
INFO:root:is_blocked=False, is_in_progress=False
INFO:root:No existing issue is being processed. Processing the next issue in the backlog...
INFO:root:Repo name: agapebondservant/industrial-data-project
INFO:root:Returning issue #1269 with status 'Backlog'
INFO:root:Repo name: agapebondservant/industrial-data-project
INFO:root:Moving next issue from <Backlog> to <In progress>...
INFO:root:Moving top issue from 'Backlog' to 'In progress'...
INFO:root:Repo name: agapebondservant/industrial-data-project
INFO:root:Repo name: agapebondservant/industrial-data-project
INFO:root:Returning issue #1269 with status 'Backlog'
INFO:root:Updating issue status to 'In progress'...
INFO:root:Successfully moved issue #1269 from 'Backlog' to 'In progress'
INFO:root:Kicking off i

File not found: /Users/tolaawofolu/eclipse-workspace/ai/usaf/workflows/examples/config/agents.yaml

File not found: /Users/tolaawofolu/eclipse-workspace/ai/usaf/workflows/examples/config/tasks.yaml

ERROR:crewai.flow.flow:Error executing listener should_sprint_continue: 'plan'


KeyError: 'plan'